# Demo program for ALBATROSS
ALBATROSS: Automated Li-ion BAttery Testing RObot SyStem

## Import module and set up

In [1]:
import sys
import time
from time import sleep
import datetime

# connect with PLC
from opcua import Client, ua
opc_client = Client("opc.tcp://192.168.1.100:4840")
opc_client.connect()

Requested session timeout to be 3600000ms, got 60000ms instead


In [2]:
# connect with Neware potentiostat
import socket

sockethost = "127.0.0.1"
socketport = 502

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((sockethost, socketport))
client_socket.settimeout(2)

def newaresend(cmd: str, default_len = 768):
    cmd_len = len(cmd)
    sendmsg = ' '*(default_len-cmd_len) + cmd
    client_socket.send(sendmsg.encode("utf-8"))

def newarerecv():
    return client_socket.recv(20000).decode("utf-8")

def neware_start(subid = 1, chlid = 1, file = "pause test", 
                 savedir = "C:/NewareData", savename = "TEST",
                 backup = "C:/NewareData/backup", type = 0):
    """
    subid: cycling device id (ex. 1 for 1-1 ~ 1-8 channel)
    chlid: channel id (ex. 1 for 1-1, 2-1, ... channel)
    file: cycling protocol file name (ex. pause test)
    backup: backup data file save path
    type: type of backup file (0: NDA, 1: excel)
    """
    newaresend('''<?xml version="1.0" encoding="UTF-8" ?> 
<bts version="1.0"> 
 <cmd>start</cmd> 
 <list count = "1"> 
  <start ip="192.168.1.2" devtype="25" devid="40" subdevid="{}" chlid="{}" >C:/NewareData/{}.xml</start> 
<backup backupdir="{}" remotedir="{}" filenametype="2" customfilename="{}" addtimewhenrepeat="0" filetype="{}" />
 </list> 
</bts>'''.format(subid, chlid, file, savedir, backup, savename, type))
    time.sleep(1)
    returnvalue = newarerecv()
    return returnvalue

def neware_continue(subid = 1, chlid = 1):
    """
    subid: cycling device id (ex. 1 for 1-1 ~ 1-8 channel)
    chlid: channel id (ex. 1 for 1-1, 2-1, ... channel)
    """
    newaresend('''<?xml version="1.0" encoding="UTF-8" ?> 
<bts version="1.0"> 
<cmd>continue</cmd> 
<list count = "1"> 
<continue ip="192.168.1.2" devtype="25" devid="40" subdevid="{}" chlid="{}" >true</continue> 
</list> 
</bts>'''.format(subid, chlid))
    time.sleep(1)
    returnvalue = newarerecv()
    return returnvalue

def neware_check(subid = 1, chlid = 1):
    """
    subid: cycling device id (ex. 1 for 1-1 ~ 1-8 channel)
    chlid: channel id (ex. 1 for 1-1, 2-1, ... channel)
    """
    newaresend('''<?xml version="1.0" encoding="UTF-8" ?> 
<bts version="1.0"> 
<cmd>getchlstatus</cmd> 
<list count = "1"> 
<status ip="192.168.1.2" devtype="25" devid="40" subdevid="{}" chlid="{}" >true</status> 
</list> 
</bts>'''.format(subid, chlid))
    time.sleep(1)
    returnvalue = newarerecv()
    if "finish" in returnvalue:
        return "finish"
    elif "working" in returnvalue:
        return "working"
    elif "stop" in returnvalue:
        return "stop"
    elif "pause" in returnvalue:
        return "pause"
    else:
        return returnvalue

In [3]:
neware_check()

'finish'

In [4]:
# connect with OT2 liquid handler
import paramiko

class OT2:
    def __init__(self, ip_addr):
        self.keyfilename = 'C:\\Users\\HOME\\ot2_ssh_key'
        try:
            self.k = paramiko.RSAKey.from_private_key_file(self.keyfilename, "1234")
        except FileNotFoundError:
            print("No such file or directory: "+self.keyfilename)
            self.keyfilename = input("Enter correct directory and file name\n")
            self.k = paramiko.RSAKey.from_private_key_file(self.keyfilename, "1234")
        self.ssh = paramiko.SSHClient()
        self.ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        #self.ssh.connect(hostname=ip_addr, username='root', pkey=self.k)
        self.ssh.connect(hostname=ip_addr, username='root', pkey=self.k, disabled_algorithms={'pubkeys': ['rsa-sha2-256', 'rsa-sha2-512']})
        self.ssh_channel = self.ssh.invoke_shell()
        #change working directory
        self.ssh_channel.send('cd /var/lib/jupyter/notebooks\n'.encode())
        time.sleep(0.3)
        #start the startup.py file which defines all the labware and homes Otto so it is ready to go
        self.ssh_channel.send('python -i startup.py\n'.encode())
        time.sleep(0.3)
        print(self.get_output())
        self.small_tip_index = 0
        self.large_tip_index = 0
        self.odacell_well_index = 0
        
    def get_output(self):
        """
        returns the output string from Otto that was stored in the stdout of the ssh client
        """
        if self.ssh_channel.recv_ready():
            output = self.ssh_channel.recv(1024)
            return output.decode()
        if  self.ssh_channel.recv_stderr_ready():
            output = self.ssh_channel.recv_stderr(1024)
            return output.decode()

    def RawInput(self, cmd):
        """
        Directly sends a string (python command) to the python program opened in the pseudo terminal shell of the Otto ssh client.\n
        Inputs:\n
        cmd (str)->python command. The characters "backslash n" is appended to the end of the command which executes it (pressing Enter in the terminal)\n
        Output:\n
        none but probably will print output from get_output() function
        """
        self.ssh_channel.send(str(cmd+"\n").encode())
        #output_str = self.get_output()
        #if isinstance(output_str, str):
        #    print(output_str)

In [5]:
# make interlock for OT2
def safety():
    while True:
        time.sleep(0.5)
        test = ot2.get_output()
        if test == None:
            break
            
    pr("protocol.rail_lights_on")
    while True:
        time.sleep(0.5)
        test = ot2.get_output()
        try:
            if "rail_lights_on\r\nTrue\r" in test:
                break
        except:
            pass

In [6]:
# connect OT2 with main PC
ot2 = OT2("192.168.1.6")
pr = ot2.RawInput



      @@@@@    @@@@@
    @@@@          @@@@
   @@@      @@      @@@    @@@@@@   @@@@@
  @@@      @@@@      @@@   @@@@@@  &@' '@@
  @@     @@@@@@@@    &@@     @@         @@
  @@    .@@@    @    #@@     @@        @@
  @@@    @      @    @@@     @@       @@
   @@@    @@..@@    @@@      @@      @@
    @@@@          @@@@       @@     @@@@@&
      @@@@@@@@@@@@@@         ##    &@@@@@#
         (@@@@@@.


cd /var/lib/jupyter/notebooks
# cd /var/lib/jupyter/notebooks
# python -i startup.py



In [ ]:
# code for communicate with PLC
def opc_send(node: str, value: bool or int or list):
    time.sleep(0.2)
    if type(value) == bool:
        opc_client.get_node("ns=4;s={}".format(node)).set_value(ua.DataValue(ua.Variant(value, ua.VariantType.Boolean)))
    elif type(value) == int:
        opc_client.get_node("ns=4;s={}".format(node)).set_value(ua.DataValue(ua.Variant(value, ua.VariantType.Int16)))
    elif type(value) == list:
        # for our case, list is composed of float or string
        try:
            opc_client.get_node("ns=4;s={}".format(node)).set_value(ua.DataValue(ua.Variant(value, ua.VariantType.Float)))
        except:
            opc_client.get_node("ns=4;s={}".format(node)).set_value(ua.DataValue(ua.Variant(value, ua.VariantType.String)))
    time.sleep(0.2)

def opc_read(node: str):
    time.sleep(0.2)
    return opc_client.get_node("ns=4;s={}".format(node)).get_value()

In [ ]:
# conde to communicate with robot arm, xArm6
import pymodbus

from pymodbus.client import ModbusTcpClient
client = ModbusTcpClient('192.168.1.196', port=502)

def robot_check():
    return opc_client.get_node("ns=4;s=Robot_moving").get_value() or opc_client.get_node("ns=4;s=Robot_moving2").get_value()

In [7]:
# code to send order to actuators and motors using PLC
def actu_move(index, val, wait=True):
    if wait:
        while True:
            time.sleep(2)
            robot_moving = robot_check()
            if not robot_moving:
                break
                
    opc_send("ActuIndex", index)
    opc_send("ActuMove", val)
    opc_send("Open_Serial", True)
    time.sleep(0.2)

import pandas as pd
jig = pd.read_excel("C:/Users/HOME/Desktop/코인셀 지그.xlsx")
jig_position = jig.iloc[:, 1:7].values.tolist()
opc_send("Jigmove_csv", jig_position)
EIS_position = jig.iloc[0:2, 7:11].values.tolist()
opc_send("EISmove_csv", EIS_position)

def actu(index = 0, wait=True):
    """
    index = 0: origin, 1~12: each component
    can --> origin = 665, distance = 285
    cap --> origin = 665, distance = 285
    spacer --> origin = 1975, distance = 135
    """
    opc_send("CCSactu_Index", index)
    time.sleep(0.2)
    opc_send("CCSactu_Check", True)
    time.sleep(0.2)

def jigjob(job, index:int=0):
    """
    Job for the coincell jig
    job: Jig_Origin, Jig_Readypos, Jig_Insert, Jig_Retrieve, EIS_Insert, EIS_Retrieve
    * Jig: 0 ~ 47 (48 ch.) / EIS: 0 ~ 1 (2 ch.)
    index: position for Insert and Retrieve
    """
    opc_send("JOB", [job, str(index)])

def camera():
    time.sleep(2)
    opc_send("ProdTracker.Capture.bStart", True)

# make interlock for robot arm
def tcp(val: int or list = None, wait = True):
    if wait == True:
        opc_send("Robot_moving2", True)
    result = client.write_registers(48, val)
    time.sleep(2)
    cnt = 0
    while True:
        time.sleep(0.2)
        if not client.read_coils(0).bits[0]:
            cnt += 1
        if cnt >= 3:
            if client.read_holding_registers(33).registers[0] == 2:
                break
    if type(result) == pymodbus.exceptions.ModbusIOException:
        tcp(val)
    if wait == True:
        opc_send("Robot_moving2", False)

In [8]:
# write remaining pipette status inside the OT2 to PLC
# two pipette decks are used: tiprack_1, tiprack_2
def pipette():
    """
    Return: ex) "tiprack_1", 0
    """
    tiprack0 = opc_read("DataOT2.Pipette0")
    tiprack1 = opc_read("DataOT2.Pipette1")

    if tiprack0 == 0 and 0 < tiprack1 < 96:
        opc_send("DataOT2.Pipette1", tiprack1+1)
        return "tiprack_2", tiprack1

    elif tiprack0 == 96 and 0 <= tiprack1 < 96:
        opc_send("DataOT2.Pipette1", tiprack1+1)
        return "tiprack_2", tiprack1

    elif tiprack0 == 96 and tiprack1 == 96:
        print("Check tiprack status! --> tiprack_1: {}, tiprack_2: {}".format(tiprack0, tiprack1))
        renew = input("input format: int, int")
        renew0, renew1 = (int(i) for i in renew.split(","))
        opc_send("DataOT2.Pipette0", renew0)
        opc_send("DataOT2.Pipette1", renew1)
        return pipette()

    else:
        opc_send("DataOT2.Pipette0", tiprack0+1)
        return "tiprack_1", tiprack0

## Initialization

In [9]:
pr("protocol.set_rail_lights(True)")

In [10]:
# OT2 offset value
pr("tiprack_1.set_offset(x=-0.30, y=1.20, z=0.00)")
pr("tiprack_2.set_offset(x=-0.30, y=1.20, z=0.00)")
pr("plate2.set_offset(x=0.2, y=1.5, z=2.00)")
pr("wellplate.set_offset(x=0.2, y=1.1, z=0.00)")
pr("temp_labware.set_offset(x=1.5, y=1.3, z=-2)")
pr("plate11.set_offset(x=0, y=0, z=6)")
pr("plate7.set_offset(x=0, y=0, z=6)")

In [11]:
# move actuator structure to zero position
actu(0)
time.sleep(1)

# move assembly post and align actuator to zero position
actu_move(10, 0, False)
time.sleep(0.2)
actu_move(3, 0, False)
time.sleep(0.2)

# move rotary indexing tables to zero position
opc_send("Motor0_Origin", True)
opc_send("Motor1_Origin", True)

# move plate stackers to zero position
opc_send("Motor2_Origin", True)
opc_send("Motor3_Origin", True)

# set gantry system boolean values to True
opc_send("JOB_Done", True)

# set automatic crimper boolean values to True
opc_send("Crimper_Done", True)

## Dosing preparation

Plate 7 <br>
A1: 1M LiPF6 with EC:EMC 3:7 VC 2wt% <br>

In [17]:
plate_info = "plate7"
dosing_list = ["A1" for i in range(3)]

In [18]:
dosing_list

['A1', 'A1', 'A1']

## Parameters for Neware potentiostat

In [19]:
protocol = "251128_demo"
savedir = 'C:/NewareData/demo/'
backup = 'C:/NewareData/demo/remote_dir/'

## Move plate

In [20]:
# elevate plate stackers
opc_send("Motor2_Up", True)
opc_send("Motor3_Up", True)
time.sleep(3)

# move plate with robot arm
tcp([5])

# elevate plate stacker
opc_send("Motor2_Up", True)

# move plates with robot arm
tcp([7, 6])

# align plate in actuator structure
opc_send("CCSalign_Check", True)

## Start assembly process

In [21]:
n = len(dosing_list)
ccs = 0 # starting position of can, cap, spacer plate
jig_pos = 0 # starting position of cycling jigs

In [22]:
# if this is the first cell to assemble, the automation sequence should be different
first_check = True

for i in range(n): 
    actu(ccs+i+1)
    
    opc_send("Motor0_Up", True)
    opc_send("Motor1_Up", True)
    
    tcp([8])
    opc_send("Assemalign_Check", True)
    camera()
    
    tcp([9])
    camera()
    opc_send("Assemalign_Check", True)
    camera()

    pr("protocol.set_rail_lights(False)")
    able_pipette = pipette()
    pr("left_tip.pick_up_tip({}.wells()[{}])".format(able_pipette[0], able_pipette[1]))
    pr("left_tip.aspirate(70, {}['{}'])".format(plate_info, dosing_list[i]))
    pr("left_tip.air_gap(20)")
    pr("left_tip.touch_tip(location = {}['{}'], radius=1, v_offset=-7, speed = 50 )".format(plate_info, dosing_list[i]))
    pr("left_tip.dispense(90, plate2['A1'])")
    pr('left_tip.drop_tip()')
    pr("protocol.set_rail_lights(True)")
    safety()
    
    tcp([10])
    camera()
    tcp([11])
    camera()
    tcp([12])
    camera()
    tcp([13])
    camera()
    tcp([14])
    camera()

    if first_check:
        first_check = False
        
        # If crimper is running, wait until it finishs
        while True:
            time.sleep(1)
            if opc_read("Crimper_Done"):
                break

        actu_move(10, 700)
        time.sleep(1)

        tcp([15])

        actu_move(10, 0)
        time.sleep(1)

        opc_send("Crimper_On", True)

        continue

    # If crimper is running, wait until it finishs
    while True:
        time.sleep(1)
        if opc_read("Crimper_Done"):
            break

    jigjob("Jig_Readypos")

    jigjob("Jig_Insert", i-1+jig_pos)

    # If jig is running, wait until it reaches the Readypos
    while True:
        time.sleep(0.5)
        if opc_read("Readypos_Done"):
            break

    tcp([16])

    # Return the Readypos check
    opc_send("Readypos_Done", False)

    # initiate Neware potentiostat automatically
    sub_id = (i-1+jig_pos)//8+1
    chl_id = (i-1+jig_pos)%8+1
    now = datetime.datetime.now().strftime("%y%m%d")
    neware_start(
        sub_id, chl_id, protocol, 
        savedir, now+"_{}-{}".format(sub_id, chl_id), 
        backup
    )
 
    actu_move(10, 700)
    time.sleep(1)
    
    tcp(15)
    
    actu_move(10, 0)
    time.sleep(1)
    
    opc_send("Crimper_On", True)

    # if it is final cell to assemble, move it to jig
    if i == n-1:
        while True:
            time.sleep(1)
            if opc_read("Crimper_Done"):
                break
    
        jigjob("Jig_Readypos")
    
        jigjob("Jig_Insert", i+jig_pos)
    
        # If jig is running, wait until it reaches the Readypos
        while True:
            time.sleep(0.5)
            if opc_read("Readypos_Done"):
                break
                                                                                                                                                                                                                                                                                                
        tcp([16])
    
        # Return the Readypos check
        opc_send("Readypos_Done", False)

        # initiate Neware potentiostat automatically
        sub_id = (i+jig_pos)//8+1
        chl_id = (i+jig_pos)%8+1
        now = datetime.datetime.now().strftime("%y%m%d")
        neware_start(
            sub_id, chl_id, protocol, 
            savedir, now+"_{}-{}".format(sub_id, chl_id), 
            backup
        )

## Retrieve cell and disposal

In [32]:
# list for cycling jig to dispose the cells
dispos_list = [6, 7, 8, 9]

In [33]:
for i in dispos_list:
    jigjob("Jig_Retrieve", i)
    jigjob("Jig_Readypos")

    while True:
        time.sleep(0.5)
        if opc_read("Readypos_Done"):
            break

    tcp(20)

    opc_send("Readypos_Done", False)

# Disconnect

In [1]:
pr('protocol.home()')
pr('left_tip.drop_tip()')
pr('protocol.home()')
pr("protocol.set_rail_lights(False)")

In [24]:
actu(0)
time.sleep(1)

In [25]:
actu_move(10, 0, False)
time.sleep(0.2)
actu_move(3, 0, False)
time.sleep(0.2)

In [26]:
opc_send("Motor0_Origin", True)
opc_send("Motor1_Origin", True)
opc_send("Motor2_Origin", True)
opc_send("Motor3_Origin", True)

In [27]:
tcp([2])

In [2]:
opc_client.disconnect()

In [33]:
ot2.ssh.close()

In [34]:
client_socket.close()